In [27]:
%%bigquery df --project isb-cgc-notebook-dev
WITH rna as (
    SELECT
       case_barcode,
       aliquot_barcode,
       gene_name,
       Ensembl_gene_id_v,
       fpkm_uq_unstranded,
       sample_type_name,
       project_short_name
    FROM `isb-cgc-bq.TCGA.RNAseq_hg38_gdc_current`
    WHERE gene_type = 'protein_coding'),
mean_expr as (
  SELECT * FROM (
    SELECT
      rna.Ensembl_gene_id_v,
      VARIANCE(rna.fpkm_uq_unstranded) var_fpkm
    FROM rna
    WHERE rna.sample_type_name = 'Solid Tissue Normal'
    GROUP BY rna.Ensembl_gene_id_v)
  ORDER BY var_fpkm DESC
  LIMIT 50)
SELECT case_barcode, aliquot_barcode, gene_name, fpkm_uq_unstranded, sample_type_name, project_short_name
FROM mean_expr
INNER JOIN rna on mean_expr.Ensembl_gene_id_v = rna.Ensembl_gene_id_v
WHERE rna.project_short_name IN ("TCGA-BRCA", "TCGA-LUAD", "TCGA-KIRC")

Query is running:   0%|          |

Downloading:   0%|          |

In [28]:
df

,case_barcode,aliquot_barcode,gene_name,fpkm_uq_unstranded,sample_type_name,project_short_name
0,TCGA-GM-A2DI,TCGA-GM-A2DI-01A-31R-A18M-07,MT-ATP6,1869.1791,Primary Tumor,TCGA-BRCA
1,TCGA-GM-A2DI,TCGA-GM-A2DI-01A-31R-A18M-07,MT-ND6,1206.0260,Primary Tumor,TCGA-BRCA
2,TCGA-GM-A2DI,TCGA-GM-A2DI-01A-31R-A18M-07,S100A9,14.2254,Primary Tumor,TCGA-BRCA
3,TCGA-GM-A2DI,TCGA-GM-A2DI-01A-31R-A18M-07,SMR3B,0.4817,Primary Tumor,TCGA-BRCA
4,TCGA-GM-A2DI,TCGA-GM-A2DI-01A-31R-A18M-07,S100A8,2.1551,Primary Tumor,TCGA-BRCA
...,...,...,...,...,...,...
122245,TCGA-75-6206,TCGA-75-6206-01A-11R-1755-07,TG,0.0225,Primary Tumor,TCGA-LUAD
122246,TCGA-75-6206,TCGA-75-6206-01A-11R-1755-07,FGB,0.0572,Primary Tumor,TCGA-LUAD
122247,TCGA-75-5146,TCGA-75-5146-01A-01R-1628-07,APOA1,0.2976,Primary Tumor,TCGA-LUAD
122248,TCGA-75-6207,TCGA-75-6207-01A-11R-1755-07,AMBP,0.1165,Primary Tumor,TCGA-LUAD


In [29]:
from google.cloud import bigquery
project, dataset, table = 'isb-cgc-notebook-dev', 'scratch', 'projects_3_genes_50'
client = bigquery.Client(project=project)
job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")
job = client.load_table_from_dataframe( df, client.dataset(dataset).table(table), job_config=job_config )
job.result()

LoadJob<project=isb-cgc-notebook-dev, location=US, id=ee28e692-28fd-4cd5-a6a2-d7d4cde470c9>

In [30]:
print("select case_barcode, aliquot_barcode,")
for gene in set(df['gene_name']):
    print(f"\tMAX(IF(gene_name = '{gene}', fpkm_uq_unstranded, NULL)) AS {gene.replace('-', '_')},")
print(f"""from `{project}.{dataset}.{table}`
group by aliquot_barcode""")

select case_barcode, aliquot_barcode,
	MAX(IF(gene_name = 'SFTPA1', fpkm_uq_unstranded, NULL)) AS SFTPA1,
	MAX(IF(gene_name = 'MT-ND6', fpkm_uq_unstranded, NULL)) AS MT_ND6,
	MAX(IF(gene_name = 'KRT6A', fpkm_uq_unstranded, NULL)) AS KRT6A,
	MAX(IF(gene_name = 'TG', fpkm_uq_unstranded, NULL)) AS TG,
	MAX(IF(gene_name = 'MT-CO2', fpkm_uq_unstranded, NULL)) AS MT_CO2,
	MAX(IF(gene_name = 'SMR3B', fpkm_uq_unstranded, NULL)) AS SMR3B,
	MAX(IF(gene_name = 'TFF1', fpkm_uq_unstranded, NULL)) AS TFF1,
	MAX(IF(gene_name = 'MT-ND4', fpkm_uq_unstranded, NULL)) AS MT_ND4,
	MAX(IF(gene_name = 'FGG', fpkm_uq_unstranded, NULL)) AS FGG,
	MAX(IF(gene_name = 'MT-CO3', fpkm_uq_unstranded, NULL)) AS MT_CO3,
	MAX(IF(gene_name = 'MT-ND4L', fpkm_uq_unstranded, NULL)) AS MT_ND4L,
	MAX(IF(gene_name = 'FGA', fpkm_uq_unstranded, NULL)) AS FGA,
	MAX(IF(gene_name = 'RBP4', fpkm_uq_unstranded, NULL)) AS RBP4,
	MAX(IF(gene_name = 'AMBP', fpkm_uq_unstranded, NULL)) AS AMBP,
	MAX(IF(gene_name = 'HP', fpkm_uq_unstranded

In [33]:
%%bigquery df2 --project isb-cgc-notebook-dev
select case_barcode, aliquot_barcode,
	MAX(IF(gene_name = 'SFTPA1', fpkm_uq_unstranded, NULL)) AS SFTPA1,
	MAX(IF(gene_name = 'MT-ND6', fpkm_uq_unstranded, NULL)) AS MT_ND6,
	MAX(IF(gene_name = 'KRT6A', fpkm_uq_unstranded, NULL)) AS KRT6A,
	MAX(IF(gene_name = 'TG', fpkm_uq_unstranded, NULL)) AS TG,
	MAX(IF(gene_name = 'MT-CO2', fpkm_uq_unstranded, NULL)) AS MT_CO2,
	MAX(IF(gene_name = 'SMR3B', fpkm_uq_unstranded, NULL)) AS SMR3B,
	MAX(IF(gene_name = 'TFF1', fpkm_uq_unstranded, NULL)) AS TFF1,
	MAX(IF(gene_name = 'MT-ND4', fpkm_uq_unstranded, NULL)) AS MT_ND4,
	MAX(IF(gene_name = 'FGG', fpkm_uq_unstranded, NULL)) AS FGG,
	MAX(IF(gene_name = 'MT-CO3', fpkm_uq_unstranded, NULL)) AS MT_CO3,
	MAX(IF(gene_name = 'MT-ND4L', fpkm_uq_unstranded, NULL)) AS MT_ND4L,
	MAX(IF(gene_name = 'FGA', fpkm_uq_unstranded, NULL)) AS FGA,
	MAX(IF(gene_name = 'RBP4', fpkm_uq_unstranded, NULL)) AS RBP4,
	MAX(IF(gene_name = 'AMBP', fpkm_uq_unstranded, NULL)) AS AMBP,
	MAX(IF(gene_name = 'HP', fpkm_uq_unstranded, NULL)) AS HP,
	MAX(IF(gene_name = 'APOA1', fpkm_uq_unstranded, NULL)) AS APOA1,
	MAX(IF(gene_name = 'PGC', fpkm_uq_unstranded, NULL)) AS PGC,
	MAX(IF(gene_name = 'MT-CYB', fpkm_uq_unstranded, NULL)) AS MT_CYB,
	MAX(IF(gene_name = 'FGB', fpkm_uq_unstranded, NULL)) AS FGB,
	MAX(IF(gene_name = 'APOC3', fpkm_uq_unstranded, NULL)) AS APOC3,
	MAX(IF(gene_name = 'SERPINA1', fpkm_uq_unstranded, NULL)) AS SERPINA1,
	MAX(IF(gene_name = 'MT-ND2', fpkm_uq_unstranded, NULL)) AS MT_ND2,
	MAX(IF(gene_name = 'KRT13', fpkm_uq_unstranded, NULL)) AS KRT13,
	MAX(IF(gene_name = 'SEMG2', fpkm_uq_unstranded, NULL)) AS SEMG2,
	MAX(IF(gene_name = 'SFTPA2', fpkm_uq_unstranded, NULL)) AS SFTPA2,
	MAX(IF(gene_name = 'S100A8', fpkm_uq_unstranded, NULL)) AS S100A8,
	MAX(IF(gene_name = 'FTL', fpkm_uq_unstranded, NULL)) AS FTL,
	MAX(IF(gene_name = 'APOH', fpkm_uq_unstranded, NULL)) AS APOH,
	MAX(IF(gene_name = 'S100A9', fpkm_uq_unstranded, NULL)) AS S100A9,
	MAX(IF(gene_name = 'GKN1', fpkm_uq_unstranded, NULL)) AS GKN1,
	MAX(IF(gene_name = 'SEMG1', fpkm_uq_unstranded, NULL)) AS SEMG1,
	MAX(IF(gene_name = 'SFTPC', fpkm_uq_unstranded, NULL)) AS SFTPC,
	MAX(IF(gene_name = 'ACTA1', fpkm_uq_unstranded, NULL)) AS ACTA1,
	MAX(IF(gene_name = 'MT-CO1', fpkm_uq_unstranded, NULL)) AS MT_CO1,
	MAX(IF(gene_name = 'LIPF', fpkm_uq_unstranded, NULL)) AS LIPF,
	MAX(IF(gene_name = 'HTN3', fpkm_uq_unstranded, NULL)) AS HTN3,
	MAX(IF(gene_name = 'ORM1', fpkm_uq_unstranded, NULL)) AS ORM1,
	MAX(IF(gene_name = 'APOA2', fpkm_uq_unstranded, NULL)) AS APOA2,
	MAX(IF(gene_name = 'MT-ND1', fpkm_uq_unstranded, NULL)) AS MT_ND1,
	MAX(IF(gene_name = 'ALB', fpkm_uq_unstranded, NULL)) AS ALB,
	MAX(IF(gene_name = 'GAST', fpkm_uq_unstranded, NULL)) AS GAST,
	MAX(IF(gene_name = 'SPRR3', fpkm_uq_unstranded, NULL)) AS SPRR3,
	MAX(IF(gene_name = 'SPRR1A', fpkm_uq_unstranded, NULL)) AS SPRR1A,
	MAX(IF(gene_name = 'KRT4', fpkm_uq_unstranded, NULL)) AS KRT4,
	MAX(IF(gene_name = 'MT-ND5', fpkm_uq_unstranded, NULL)) AS MT_ND5,
	MAX(IF(gene_name = 'MT-ATP6', fpkm_uq_unstranded, NULL)) AS MT_ATP6,
	MAX(IF(gene_name = 'SAA1', fpkm_uq_unstranded, NULL)) AS SAA1,
	MAX(IF(gene_name = 'MT-ND3', fpkm_uq_unstranded, NULL)) AS MT_ND3,
	MAX(IF(gene_name = 'STATH', fpkm_uq_unstranded, NULL)) AS STATH,
	MAX(IF(gene_name = 'LCN1', fpkm_uq_unstranded, NULL)) AS LCN1,
from `isb-cgc-notebook-dev.scratch.projects_3_genes_50`
group by case_barcode, aliquot_barcode

Query is running:   0%|          |

Downloading:   0%|          |

In [34]:
df2

,case_barcode,aliquot_barcode,SFTPA1,MT_ND6,KRT6A,TG,MT_CO2,SMR3B,TFF1,MT_ND4,...,GAST,SPRR3,SPRR1A,KRT4,MT_ND5,MT_ATP6,SAA1,MT_ND3,STATH,LCN1
0,TCGA-GM-A2DI,TCGA-GM-A2DI-01A-31R-A18M-07,0.0959,1206.0260,0.4554,0.0739,3310.7722,0.4817,268.7418,2712.7160,...,0.0000,0.0272,0.0000,0.5721,1039.7199,1869.1791,18.4350,1474.2010,0.0000,0.0000
1,TCGA-GM-A2DK,TCGA-GM-A2DK-01A-21R-A180-07,0.0000,9595.0557,0.1979,0.1373,17868.0871,0.0000,535.9416,23090.7174,...,0.0000,0.0000,0.0000,0.0000,6989.2436,12400.3680,205.8030,9693.1737,0.0000,0.0881
2,TCGA-GM-A2DM,TCGA-GM-A2DM-01A-11R-A180-07,0.0000,1579.5919,0.5623,0.0191,3592.4223,0.0280,171.0955,3435.2715,...,0.0000,0.0000,0.0000,0.0000,948.9857,2628.8546,1.7720,2516.3278,0.0000,0.0245
3,TCGA-GM-A2DL,TCGA-GM-A2DL-01A-11R-A18M-07,0.0000,1275.2921,0.0082,0.0300,15002.4590,0.0000,550.2746,10797.2296,...,0.0000,0.0166,0.9732,0.5646,1424.0308,9446.1279,65.1341,8601.7312,0.0000,0.0303
4,TCGA-GM-A2DN,TCGA-GM-A2DN-01A-11R-A180-07,0.0466,1284.5739,0.8458,0.0599,4705.7052,0.0688,720.1908,4111.0315,...,0.0000,0.0000,0.0000,0.0066,1025.5133,2937.8888,76.0135,3380.5907,0.0000,0.0483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440,TCGA-75-6207,TCGA-75-6207-01A-11R-1755-07,179.4697,482.9784,0.4229,0.0278,2219.4611,0.0000,0.0000,1780.0101,...,0.0000,0.0307,0.0000,4.8622,378.7804,735.8862,3.2059,1016.3614,0.0103,0.0840
2441,TCGA-75-6205,TCGA-75-6205-01A-11R-1755-07,26.1532,446.3406,347.7683,0.1225,2306.3178,0.0000,0.0000,2133.3256,...,0.0000,0.0592,0.0000,0.1037,394.2889,1434.5226,40.4597,1608.7481,0.0000,0.1621
2442,TCGA-75-6203,TCGA-75-6203-01A-11R-1755-07,7708.2748,5371.3020,0.4419,0.0973,8898.2213,0.0505,8.5947,9392.3921,...,0.0752,0.0000,0.0000,13.4467,3250.8890,5749.1978,0.7875,5040.0267,0.0000,0.0000
2443,TCGA-75-6206,TCGA-75-6206-01A-11R-1755-07,204.0923,2149.8267,0.3632,0.0225,2525.6637,0.0000,0.0000,3246.4224,...,0.0000,0.0000,0.0000,0.0435,1423.4626,1902.1450,0.0000,1538.7877,0.0000,0.0000


In [35]:
project, dataset, table = 'isb-cgc-notebook-dev', 'scratch', 'pivot_projects_3_genes_50'
client = bigquery.Client(project=project)
job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")
job = client.load_table_from_dataframe( df2, client.dataset(dataset).table(table), job_config=job_config )
job.result()

LoadJob<project=isb-cgc-notebook-dev, location=US, id=8792f611-4a9f-47e4-a87a-9561dce4361f>

In [39]:
%%bigquery df3 --project isb-cgc-notebook-dev
select a.*, b.disease_type EXCEPT a.case_barcode, a.aliquot_barcode from `isb-cgc-notebook-dev.scratch.pivot_projects_3_genes_50` a
join `isb-cgc-bq.TCGA.clinical_gdc_current` b
on a.case_barcode = b.submitter_id

Executing query with job ID: b71a2870-0034-4c39-b771-3bc967829ac1
Query executing: 1.76s


ERROR:
 400 EXCEPT must be followed by ALL, DISTINCT, or "(" at [1:28]; reason: invalidQuery, location: query, message: EXCEPT must be followed by ALL, DISTINCT, or "(" at [1:28]

Location: US
Job ID: b71a2870-0034-4c39-b771-3bc967829ac1



In [37]:
df3

,case_barcode,aliquot_barcode,SFTPA1,MT_ND6,KRT6A,TG,MT_CO2,SMR3B,TFF1,MT_ND4,...,SPRR3,SPRR1A,KRT4,MT_ND5,MT_ATP6,SAA1,MT_ND3,STATH,LCN1,disease_type
0,TCGA-BH-A0B2,TCGA-BH-A0B2-01A-11R-A10J-07,0.0729,614.1199,0.1357,0.3530,3596.7109,0.2870,94.6704,2608.5840,...,0.0000,0.0735,0.0483,587.5943,1802.1687,71.2208,1739.6519,0.0000,0.0755,Ductal and Lobular Neoplasms
1,TCGA-49-4507,TCGA-49-4507-01A-01R-1206-07,6.9215,1175.4454,663.9344,0.0928,9315.1259,0.0000,231.4073,6968.4986,...,0.0000,0.3960,0.0310,878.2381,6388.0874,70.2512,5097.8259,0.0000,0.3388,Adenomas and Adenocarcinomas
2,TCGA-64-5774,TCGA-64-5774-01A-01R-1628-07,71.0701,560.9934,0.3099,0.0257,2791.9598,0.0000,68.7825,2237.9338,...,0.0000,0.2652,0.0249,561.0846,1116.8325,21.4115,997.2381,0.0000,0.0907,Adenomas and Adenocarcinomas
3,TCGA-50-5072,TCGA-50-5072-01A-21R-1858-07,980.2412,1000.5117,7.0120,0.0460,5531.5602,0.0000,41.9731,6869.7451,...,0.0618,0.0000,0.5498,807.2328,3603.9948,6.4948,3488.0840,0.0000,0.1129,Adenomas and Adenocarcinomas
4,TCGA-50-5941,TCGA-50-5941-01A-11R-1755-07,1004.4695,433.0549,16.9308,0.0607,2886.9915,0.0000,0.7496,2101.1359,...,0.1280,2.2764,0.3632,389.8630,1438.6632,254.1216,1863.8034,0.0000,0.0390,Adenomas and Adenocarcinomas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440,TCGA-44-6146,TCGA-44-6146-01A-11R-A278-07,1021.5669,10.9579,0.3068,0.0367,427.2149,0.0000,5798.7655,57.1151,...,0.0283,0.0000,1.0069,23.9130,67.4620,0.3448,412.4885,0.0000,0.0000,"Cystic, Mucinous and Serous Neoplasms"
2441,TCGA-44-6146,TCGA-44-6146-01B-04R-A277-07,443.9844,201.4990,0.3274,0.1434,1941.5452,0.0990,725.1515,1465.9507,...,0.0633,0.0000,0.2220,764.4205,1026.5614,0.0643,617.1548,0.0426,0.0000,"Cystic, Mucinous and Serous Neoplasms"
2442,TCGA-44-6148,TCGA-44-6148-11A-01R-1858-07,5594.2178,4533.6740,0.0992,0.0615,5537.7263,0.0000,0.2951,6777.6381,...,0.0000,0.0000,12.4128,2632.3479,4233.7306,10.9534,4107.6623,0.0000,0.0789,"Cystic, Mucinous and Serous Neoplasms"
2443,TCGA-44-6148,TCGA-44-6148-01A-11R-1755-07,1029.8989,1688.1193,0.0443,0.1166,5102.0850,2.2893,43.2522,5882.8832,...,0.0000,0.0000,6.6660,1260.4423,3324.3950,9.9054,4444.2715,0.0000,0.0000,"Cystic, Mucinous and Serous Neoplasms"


In [38]:
project, dataset, table = 'isb-cgc-notebook-dev', 'scratch', 'pivot_projects_3_genes_50_adddiag'
client = bigquery.Client(project=project)
job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")
job = client.load_table_from_dataframe( df3, client.dataset(dataset).table(table), job_config=job_config )
job.result()

LoadJob<project=isb-cgc-notebook-dev, location=US, id=1e338303-0a78-42f7-a928-ae05b3c3bfc4>